In [3]:
import cv2
import time
import numpy as np
import mediapipe as mp
import winsound  # For beep sound on Windows

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Define eye landmarks
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
LEFT_EYE = [33, 160, 158, 133, 153, 144]

def distance(point_1, point_2):
    return np.linalg.norm(np.array(point_1) - np.array(point_2))

def get_ear(landmarks, eye_indices, frame_width, frame_height):
    coords = []
    for i in eye_indices:
        lm = landmarks[i]
        coords.append((lm.x * frame_width, lm.y * frame_height))
    
    P2_P6 = distance(coords[1], coords[5])
    P3_P5 = distance(coords[2], coords[4])
    P1_P4 = distance(coords[0], coords[3])
    
    ear = (P2_P6 + P3_P5) / (2.0 * P1_P4)
    return ear

def main():
    cap = cv2.VideoCapture(0)
    drowsy_frames = 0
    alert_time = 2  # seconds
    start_time = time.time()

    with mp_face_mesh.FaceMesh(max_num_faces=1) as face_mesh:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_frame)

            if results.multi_face_landmarks:
                landmarks = results.multi_face_landmarks[0].landmark
                ear = (get_ear(landmarks, LEFT_EYE, frame.shape[1], frame.shape[0]) +
                       get_ear(landmarks, RIGHT_EYE, frame.shape[1], frame.shape[0])) / 2.0
                
                # Display EAR value on the frame
                cv2.putText(frame, f'EAR: {ear:.2f}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                if ear < 0.2:  # Threshold for drowsiness
                    drowsy_frames += 1
                    if drowsy_frames > 20:  # Adjust this value for sensitivity
                        cv2.putText(frame, "DROWSINESS DETECTED!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        if time.time() - start_time > alert_time:
                            winsound.Beep(1000, 1000)  # Beep sound
                else:
                    drowsy_frames = 0
                    start_time = time.time()  # Reset timer

            cv2.imshow('Drowsiness Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()